# Optuna

In [ ]:
import optuna
import torch
import os
from torch.optim.lr_scheduler import ReduceLROnPlateau
from gcn_model import GCN
from train_eval import train, evaluate
from dataset_utils import get_dataloaders_optuna


In [ ]:
#%%
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)


In [ ]:
#%%
def get_dataloaders(batch_size):
    return get_dataloaders_optuna(batch_size)


In [ ]:
#%%
def objective(trial):
    # --- Hiperparámetros a optimizar ---
    num_layers = trial.suggest_int("num_layers", 10, 30)
    hidden_dim = trial.suggest_categorical("hidden_dim", [32, 64, 128, 256, 512])
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.2)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    # --- Cargar datos ---
    train_loader, val_loader, test_loader, input_dim = get_dataloaders(batch_size=batch_size)

    # --- Crear modelo ---
    model = GCN(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        output_dim=1,
        num_layers=num_layers,
        use_dropout=True,
        dropout_rate=dropout_rate,
        use_batchnorm=False,
        use_residual=False
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # --- Scheduler con ReduceLROnPlateau ---
    scheduler = ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=10,
        min_lr=1e-6
    )

    # --- Entrenamiento + Evaluación en validación ---
    for epoch in range(50):
        train(model, train_loader, optimizer, device)
        val_loss, _, _, _ = evaluate(model, val_loader, device, error_threshold=1.0)
        scheduler.step(val_loss)

    return val_loss  # Métrica a minimizar


## Estudio

In [ ]:
#%%
# Ejecuta este bloque y haz clic en el enlace generado para abrir el panel de control de Optuna
!pip install optuna-dashboard

import optuna_dashboard

# Ejecutar dashboard en segundo plano
optuna_dashboard.run_server("sqlite:///gcn_optuna_3.db", port=8080, host="0.0.0.0", no_browser=True)


In [ ]:
#%%
study = optuna.create_study(
    direction="minimize",
    study_name="gcn_study_3",
    storage="sqlite:///gcn_optuna_3.db",  # Persistente para dashboard
    load_if_exists=True
)

study.optimize(objective, n_trials=100)


In [ ]:
#%%
print("Mejor conjunto de hiperparámetros encontrados:")
for key, val in study.best_trial.params.items():
    print(f"{key}: {val}")


In [ ]:
#%%
import optuna.visualization.matplotlib as vis

fig1 = vis.plot_optimization_history(study)
fig1.show()

fig2 = vis.plot_param_importances(study)
fig2.show()
